In [ ]:
%pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [5]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [6]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team.",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working on providing support to {customer}, a super"
        "customer for your company."
        "You need to make sure that you provide the best support!"
        "Make sure to provide full complete answers and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the best support quality assurance in your team.",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working with your team on a request from {customer} "
        "ensuring that the support representative is providing the best support possible.\n"
        "You need to make sure that the support representative is providing full complete answers and make no assumptions."
    ),
    # allow_delegation=True, true by default.
    verbose=True
)

In [9]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# several more tools available like Load Customer data, tap into previous conversations load data from CRM, checking existing reports/feature requests/tickets etc

In [10]:
search_tool = SerperDevTool()
scrap_tool = ScrapeWebsiteTool() # can be used to scrap any URL

docs_scrap_tool = ScrapeWebsiteTool(        # can only be used to scrap from given url
    website_url="https:\\docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off"
)

In [11]:
# Two was to give agents tools. Can be done at Agent level or Task level.

In [12]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n {inquiry}\n\n {person} from {customer} is"
        "the one that reached out. Make sure to use everything you know to provide the best support possible."
        "You must strive to provide a complete and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the constomer's inquiry that addresses all aspects of their question.\n"
        "The response should include references to everything you used to find the answer, including external "
        "data or solutions. Ensure the answer is complete, leaving no questions unanswered, and maintain a "
        "helpful and friendly tone throughout. "
    ),
    tools=[docs_scrap_tool],
    agent=support_agent,
)

In [13]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative. Ensure that the answer is comprehensive, "
        "accurate and adheres high-quality standards expected for customer support.\n Verify that all parts of the "
        "customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to find the information, ensuring the response is well-supported "
        "and leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed and informative response ready to be sent to customer.\n This response should fully "
        "address the customer's inquiry, incorporating all relevant feedback and improvements.\n Don't be too "
        "formal, we are a chill and cool company but maintain a professional and friendly tone throughout."
    ),
    agent = support_quality_assurance_agent,
)

In [15]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True  # makes all three memories availble to the crew. SHort term, long term and entity. 
)

2024-07-13 04:47:43,428 - 10620 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [16]:
inputs = {
    "customer":"DeapLearningAI",
    "person":"Andrew Ng",
    "inquiry":"I need helpw with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?"
}

In [ ]:
result = crew.kickoff(inputs=inputs)